# 0.Prep

## 0.1 Import Libs

In [ ]:
import pandas as pd
import numpy as np
import cv2    #OpenCV基于BSD许可(开源)发行的跨平台计算机视觉库
from glob import glob    #glob.golb：匹配路径文件名

import sklearn
from sklearn.model_selection import GroupKFold, StratifiedKFold    #scikit-learn
from sklearn.metrics import roc_auc_score, log_loss
from sklearn import metrics
from sklearn.metrics import log_loss

from skimage import io    #Scikit-Image基于python脚本语言开发的数字图片处理包；子模块io读取、保存和显示图片或视频
import os
from datetime import datetime
import time
import random
import torchvision
from torchvision import transforms
from tqdm import tqdm    #进度条

import matplotlib.pyplot as plt

import torch
from torch import nn
from torch.utils.data import Dataset,DataLoader
from torch.utils.data.sampler import SequentialSampler, RandomSampler
from torch.cuda.amp import autocast, GradScaler

import warnings
import joblib

from scipy.ndimage.interpolation import zoom

## 0.2 Configuration

In [ ]:
CFG = {    #configuration
    'fold_num': 12,
    'seed': 719,
    'model_arch': 'tf_efficientnet_b3_ns',
    'img_size': 384,
    'epochs': 120,
    'train_bs': 28,
    'valid_bs': 32,
    'lr': 1e-2,
    'num_workers': 5,
    'accum_iter': 1,
    'verbose_step': 2,
    'device': 'cuda:0',
    'tta': 10,
    'used_epochs': [6,7,8,9],
    'weights': [1,1,1,1]
}

## 0.3 Helper function

In [ ]:
def all_seed(seed):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = True

In [ ]:
def get_img(path):
    im_bgr = cv2.imread(path)
    im_rgb = im_bgr[:, :, ::-1]
    return im_rgb

## 0.4 Data Profile

Image

In [ ]:
#显示图像
img = get_img('../input/cassava-leaf-disease-classification/train_images/1000015157.jpg')
plt.imshow(img)
plt.show()

csv

In [ ]:
#读csv文件
train = pd.read_csv('../input/cassava-leaf-disease-classification/train.csv')

In [ ]:
#csv文件前五项
train.head()

In [ ]:
#训练集数据总况
train.label.value_counts()

Sample of submission

In [ ]:
#要提交的csv文件样例
sample_submission = pd.read_csv('../input/cassava-leaf-disease-classification/sample_submission.csv')
sample_submission.head()

# 1. Dataset

In [ ]:
class CassavaDataset(Dataset):    #继承类(基类：torch.utils.data.Dataset)
    
    def __init__(self, df, data_root, transforms = None, output_label = True):    #类的初始化方法
        '''
        Arguments:
            self: 类的实例
            df: pandas数据帧(DataFrame)
            data_root: 数据根目录
            transforms: 数据增强
            output_label: 输出标记

        Returns:
            None
        '''
        super().__init__()
        self.df = df.reset_index(drop = True).copy()
        self.transforms = transforms
        self.data_root = data_root
        self.output_label = output_label
        
    def __len__(self):    #返回数据集大小
        '''
        Arguments:
            self: 类的实例
        
        Returns:
            self.df.shape[0]: 数据集大小
        '''
        return self.df.shape[0]
    
    def __getitem__(self, index: int):    #按索引取出数据，并预处理
        '''
        Arguments:
            self: 类的定义
            index: 要取数据的索引
        
        Returns:
            img: 图像
            target: 当output_label为True，则输出图像相应标记(label)
        '''
        #获取图像
        #pandas.iloc基于整数的索引方式，第index位的图像id
        path = "{}/{}".format(self.data_root, self.df.iloc[index]['image_id'])
        img = get_img(path)
        
        #图像预处理
        if self.transforms:
            img = self.transforms(image = img)['image']
            
        #获取标记(label)
        if self.output_label:
            target = self.df.iloc[index]['label']
        
        #返回值
        if self.output_label:
            return img, target
        else:
            return img

## 2.Augmentation

In [ ]:
from albumentations import (
    HorizontalFlip, VerticalFlip, Transpose, ShiftScaleRotate, 
    HueSaturationValue,RandomResizedCrop, RandomBrightnessContrast, 
    Compose, Normalize, Cutout, CoarseDropout, CenterCrop, Resize
)
#albumentations: 图片数据增强库

from albumentations.pytorch import ToTensorV2

In [ ]:
def get_train_transforms():
    '''
    Arguments:
        None
    
    Returns:
        Compose: albumentations.Compose(),串联多个图片变换的操作
            RandomResizedCrop: 图像随机裁剪为不同大小和宽高比，然后缩放为制定的大小
            Transpose: 转置
            HorizontalFlip: Y轴水平翻转
            VerticalFlip: X轴垂直翻转
            ShiftScaleRotate: 随机应用仿射变换：平移，缩放和旋转
            HueSaturationValue: 色调饱和度值
            RandomBrightnessContrast: 随机亮度对比度
            Normalize: 归一化
            CoarseDropout: 在图像上生成矩形区域
            Cutout: 在图像中生成正方形区域
            ToTensorV2: 张量转换     
    '''
    return Compose([
        RandomResizedCrop(CFG['img_size'], CFG['img_size']),
        Transpose(p=0.5),
        HorizontalFlip(p=0.5),
        VerticalFlip(p=0.5),
        ShiftScaleRotate(p=0.5),
        HueSaturationValue(hue_shift_limit=0.2, sat_shift_limit=0.2, val_shift_limit=0.2, p=0.5),
        RandomBrightnessContrast(brightness_limit=(-0.1,0.1), contrast_limit=(-0.1, 0.1), p=0.5),
        Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225], max_pixel_value=255.0, p=1.0),
        CoarseDropout(p=0.5),
        Cutout(p=0.5),
        ToTensorV2(p=1.0),
    ], p=1.)

In [ ]:
def get_valid_transforms():
    '''
    Arguments:
        None
    
    Returns:
        Compose
            CenterCrop: 随机中心裁剪
            Resize: 将图像调整为给定的高度和宽度
            Normalize: 归一化
            ToTensorV2: 张量转换
    '''
    return Compose([
        CenterCrop(CFG['img_size'], CFG['img_size'], p=1.),
        Resize(CFG['img_size'], CFG['img_size']),
        Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225], max_pixel_value=255.0, p=1.0),
        ToTensorV2(p=1.0),
    ], p=1.)

In [ ]:
def get_inference_transforms():
    '''
    Arguments:
        None
    
    Returns:
        Compose: albumentations.Compose(),串联多个图片变换的操作
            RandomResizedCrop: 图像随机裁剪为不同大小和宽高比，然后缩放为制定的大小
            Transpose: 转置
            HorizontalFlip: Y轴水平翻转
            VerticalFlip: X轴垂直翻转
            HueSaturationValue: 色调饱和度值
            RandomBrightnessContrast: 随机亮度对比度
            Normalize: 归一化
            ToTensorV2: 张量转换     
    '''
    return Compose([
        RandomResizedCrop(CFG['img_size'], CFG['img_size']),
        Transpose(p=0.5),
        HorizontalFlip(p=0.5),
        VerticalFlip(p=0.5),
        HueSaturationValue(hue_shift_limit=0.2, sat_shift_limit=0.2, val_shift_limit=0.2, p=0.5),
        RandomBrightnessContrast(brightness_limit=(-0.1,0.1), contrast_limit=(-0.1, 0.1), p=0.5),
        Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225], max_pixel_value=255.0, p=1.0),
        ToTensorV2(p=1.0),
    ], p=1.)

## 3.Model

In [ ]:
import sys

package_path = '../input/pytorch-image-models/pytorch-image-models-master'
sys.path.append(package_path)

import timm

In [ ]:
class CassavaImgClassifier(nn.Module):    #继承类(torch.nn.Module)

    def __init__(self, model_arch, n_class, pretrained=False):
        '''
        Arguments:
            self:类的实例
            model_arch:模型名
            n_class: 类别数
            pretrained: 是否加载预训练模型
            
        Returns:
            None
        '''
        super().__init__()
        self.model = timm.create_model(model_arch, pretrained=pretrained)
        n_features = self.model.classifier.in_features
        self.model.classifier = nn.Linear(n_features, n_class)
        
    def forward(self, x):    #前向传递
        x = self.model(x)
        return x

**CLASS** PyTorch.nn.Linear(*in_features, out_features, bias = True*)

Applies a linear transformation to the incoming data: $y = xA^T+b$

用于设置网络中的全连接层，输入与输出都是二维张量，一般形状为`[batch_size,size]`

`in_features`指的是输入的二维张量的大小，即输入的`[batch_size, size]`中的`size`。

`out_features`指的是输出的二维张量的大小，也代表了该全连接层的神经元个数。

**efficientnet**

卷积神经网络(cnn)通常是以固定的资源成本开发，然后在更多资源加入进来时扩大规模，以达到更高精度。传统的模型缩放实践是任意增加 CNN 的深度或宽度，或使用更大的输入图像分辨率进行训练和评估。 虽然这些方法确实提高了准确性，但它们通常需要长时间的手动调优，并且仍然会经常产生次优的性能。

最近的一篇ICML文章提出了一个更有原则性的方法来扩大 CNN 的规模，从而可以获得更好的准确性和效率。该论文提出了一种新的模型缩放方法，它使用一个简单而高效的复合系数来以更结构化的方式放大 CNNs。 不像传统的方法那样任意缩放网络维度，如宽度，深度和分辨率，该论文的方法用一系列固定的尺度缩放系数来统一缩放网络维度。 通过使用这种新颖的缩放方法和AutoML技术，作者将这种模型称为 EfficientNets ，它具有最高达10倍的效率(更小、更快)。

作者系统的研究了网络深度（Depth）、宽度（Width）和分辨率（resolution)对网络性能的影响，然后提出了一个新的缩放方法--使用简单但高效的复合系数均匀地缩放深度/宽度/分辨率的所有尺寸，在MobileNets 和 ResNet上证明了有效性。

同时，使用神经架构搜索来设计新的baseline并进行扩展以获得一系列模型，称EfficientNets，比之前的ConvNets更accuracy和更efficiency。其中EfficientNet-B7实现了ImageNet的state-of-the-art，即84.4% top-1 / 97.1% top-5 accuracy，同时该模型比现存最好的ConvNets的size小8.4倍，速度快6.1倍。

摘自：
    https://zhuanlan.zhihu.com/p/67508423
    https://zhuanlan.zhihu.com/p/67834114

In [ ]:
package_path = '../input/visiontransformer-pytorch/VisionTransformer-Pytorch-main'
sys.path.append(package_path)

from vision_transformer_pytorch import VisionTransformer

In [ ]:
class EnsembleClassifier(nn.Module):    #继承类(torch.nn.Module)
    
    def __init__(self, model_arch, n_class, pretrained=False):
        '''
        Arguments:
            self:类的实例
            model_arch:模型名
            n_class: 类别数
            pretrained: 是否加载预训练模型
            
        Returns:
            None
        '''
        super().__init__()
        self.model1 = VisionTransformer.from_name('ViT-B_16', num_classes=5)
        self.model1.load_state_dict(torch.load('../input/vit-model-1/ViT-B_16.pt'))    #预训练模型1
        self.model2 = CassavaImgClassifier(model_arch, n_class, pretrained)    #预训练模型2
        
    def forward(self, x):    #前向传递，模型1权值0.6，模型2权值0.4
        x1 = self.model1(x)
        x2 = self.model2(x)
        return 0.6 * x1 + 0.4 * x2
    
    def load(self, state_dict):
        self.model2.load_state_dict(state_dict)

## 4.Main Loop

In [ ]:
def inference_one_epoch(model, data_loader, device):
    '''
    Arguments:
        model: 
        data_loader: 
        device: 
    Returns:
        image_preds_all: 
    '''
    model.eval()    #注1

    image_preds_all = []
    
    pbar = tqdm(enumerate(data_loader), total=len(data_loader))    #进度条
        #enumerate(): 将一个可遍历的数据对象(如列表、元组或字符串)组合为一个索引序列，同时列出数据和数据下标
    for step, (imgs) in pbar:
        imgs = imgs.to(device).float()    #将数据copy到device指定的GPU上
        
        image_preds = model(imgs)   
        image_preds_all += [torch.softmax(image_preds, 1).detach().cpu().numpy()]    #注2
        
    
    image_preds_all = np.concatenate(image_preds_all, axis=0)    #对array进行拼接
    return image_preds_all

**注1：model.train()与model.eval()**

如果模型中有`BN(Batch Normalization)`和`Dropout`，需要在训练时添加`model.train()`，在测试时添加`model.eval()`。其中`model.train()`是保证`BN`用每一批数据的均值和方差，而`model.eval()`是保证`BN`用全部训练数据的均值和方差；而对于`Dropout`，`model.train()`是随机取一部分网络连接来训练更新参数，而`model.eval()`是用到了所有网络连接。

`Batch Normalization`对网络中间的每层进行归一化处理，并且使用变换重构（Batch Normalization Transform）保证每层所提取的特征分布不会被破坏。

`Dropout`能克服过拟合，该层(layer)的神经元在每次迭代训练时有概率p的可能性不参与训练

**注2：torch.softmax(image_preds, 1).detach().cpu().numpy()**

**①**`class torch.nn.Softmax(input, dim)`

对n维输入张量运用`Softmax`函数，将张量的每个元素缩放到（0,1）区间且和为1。`Softmax`函数定义如下：

$$f_i(x) = \frac{e^{(x_i-shift)}}{\Sigma^je^{(x_j-shift)}},shift = max(x_i)$$

`dim`指明维度，`dim=0`表示按列计算；`dim=1`表示按行计算

**②**`detach()`

再训练网络时可能希望保持一部分参数不变，只对其中一部分参数进行调整；或只训练部分分支网络，希望其梯度不对主网络的梯度造成影响

`detach()`阻断反向传播，返回张量(tensor)

**③**`cpu()`将数据移至CPU中

**④**`numpy()`tensor变量转numpy变量

In [ ]:
if __name__ == '__main__':    #确保只有单独运行该模块时，此表达式才成立，才可以进入此判断语法

    all_seed(CFG['seed'])
    
    folds = StratifiedKFold(n_splits=CFG['fold_num']).split(np.arange(train.shape[0]), train.label.values)
        #注1
    
    for fold, (trn_idx, val_idx) in enumerate(folds):
        
        # 首先训练 fold 0
        if fold > 0:
            break 

        print('Inference fold {} started'.format(fold))
        
        #数据集的建立(valid_ds, test_ds)
        valid_ = train.loc[val_idx,:].reset_index(drop=True)    #loc: 通过行标签索引行数据; reset_index重置索引
                  #CassavaDataset的参数：df, data_root, transforms, output_label
        valid_ds = CassavaDataset(valid_, '../input/cassava-leaf-disease-classification/train_images/', 
                                  transforms = get_inference_transforms(), output_label=False)

        test = pd.DataFrame()
        test['image_id'] = list(os.listdir('../input/cassava-leaf-disease-classification/test_images/'))
            #os.listdir: 返回指定的文件夹包含的文件或文件夹的名字的列表
        test_ds = CassavaDataset(test, '../input/cassava-leaf-disease-classification/test_images/', 
                                 transforms = get_inference_transforms(), output_label=False)
        
        #读数据
        val_loader = torch.utils.data.DataLoader(
            valid_ds, 
            batch_size = CFG['valid_bs'],
            num_workers = CFG['num_workers'],
            shuffle = False,
            pin_memory = False,
        )
        
        tst_loader = torch.utils.data.DataLoader(
            test_ds, 
            batch_size = CFG['valid_bs'],
            num_workers = CFG['num_workers'],
            shuffle = False,
            pin_memory = False,
        )

        device = torch.device(CFG['device'])
        model = EnsembleClassifier(CFG['model_arch'], train.label.nunique()).to(device)
        
        val_preds = []
        tst_preds = []
        
        #for epoch in range(CFG['epochs']-3):
        for i, epoch in enumerate(CFG['used_epochs']):    
            model.load(torch.load('../input/fork-pytorch-efficientnet-baseline-train-amp-a/{}_fold_{}_{}'.format(CFG['model_arch'], fold, epoch)))
            
            with torch.no_grad():
                for _ in range(CFG['tta']):
                    val_preds += [CFG['weights'][i]/sum(CFG['weights'])/CFG['tta']*inference_one_epoch(model, val_loader, device)]
                    tst_preds += [CFG['weights'][i]/sum(CFG['weights'])/CFG['tta']*inference_one_epoch(model, tst_loader, device)]

        val_preds = np.mean(val_preds, axis=0) 
        tst_preds = np.mean(tst_preds, axis=0) 
        
        print('fold {} validation loss = {:.5f}'.format(fold, log_loss(valid_.label.values, val_preds)))
        print('fold {} validation accuracy = {:.5f}'.format(fold, (valid_.label.values==np.argmax(val_preds, axis=1)).mean()))
        
        del model
        torch.cuda.empty_cache()

**注1**`StratifiedKFold`

① *k折交叉验证*

- 将全部训练集S分成k个不相交的子集，假设S中的训练样例个数为m，则每个子集有m/k个训练样例，相应的子集为{s1，s2，...，sk}
- 每次从分好的子集里面，拿出一个作为测试集，其他k-1个作为训练集
- 在k-1个训练集上训练出模型，再把这个模型放到测试集上，得到分类率的平均值，作为该模型或者假设函数的真实分类率
- 10折交叉验证是最常用的

② *StratifiedKFold 和 KFold 的比较*

- StratifiedKFold 分层采样交叉切分，确保训练集，测试集中各类别样本的比例与原始数据集中相同
- KFold不能整除时，每个互斥集的样本数会尽量相近

③ *StratifiedKFold参数*
- `n_splits`: 折叠次数，默认为3，至少为2
- `shuffle`: 是否在每次分割之前打乱顺序
- `random_state`: 随机种子，在shuffle==True时使用，默认使用np.random

④ *.split(X,y)*
- X:array-like,shape(n_sample,n_features)，训练数据集。
- y:array-like,shape(n_sample)，标签。
- 返回值：训练集数据的index与验证集数据的index。

In [ ]:
test['label'] = np.argmax(tst_preds, axis=1)
test.head()

In [ ]:
test.to_csv('submission.csv', index = False)